In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import lognorm, norm
import statsmodels.api as sm
from typing import Tuple
from matplotlib.pyplot import figure
from datetime import date
from math import exp

#### In integral form:

$$
    X_t - V(t, S_T) = \underbrace{\frac{X_0 - V(0, S_0)}{D_t}}_{\text{Premium component}} + \underbrace{\frac{1}{D_t} \int_0^t D_u (r_t - r) \left(V(u, S_u) - \frac{\partial V}{\partial s}(u, S_u) S_u \right)du}_{\text{Funding component}}
$$
$$
+ \underbrace{\frac{1}{D_t} \int_0^t \left(\Delta_u - \frac{\partial V}{\partial s}(u, S_u) \right) d(D_u S_u)}_{\text{Delta component}} + \underbrace{\frac{1}{2} \frac{1}{D_t} \int_0^t D_u \frac{\partial^2 V}{\partial s^2}(u, S_u) S_u^2(\sigma^2 - \sigma_t^2) du}_{\text{Gamma component}}
$$

In [ ]:
def PVPutBS(
    spotPrices: np.array,
    strike: float,
    maturity: float,
    riskFreeRate: float,
    volatility: float
) -> np.array:
    """
    Input parameters:
    spotPrices - array of spot prices
    strike - option strike
    maturity - option maturity
    riskFreeRate - current risk free rate 
    volatility - implied BS volatility
    
    Returns:
    np.array of option prices
    """

    d1 = (np.log(spotPrices / strike) + (riskFreeRate + 0.5 *
          volatility ** 2) * maturity) / (volatility * np.sqrt(maturity))
    d2 = d1 - volatility * np.sqrt(maturity)

    return spotPrices * norm.cdf(d1) - strike * np.exp(-riskFreeRate * maturity) * norm.cdf(d2)

In [ ]:
def DeltaPutBS(
    spotPrices: np.array,
    strike: float,
    maturity: float,
    riskFreeRate: float,
    volatility: float
) -> np.array:
    """
    Input parameters:
    spotPrices - array of spot prices
    strike - option strike
    maturity - option maturity
    riskFreeRate - current risk free rate 
    volatility - implied BS volatility
    
    Returns:
    np.array of option deltas
    """

    d1 = (np.log(spotPrices / strike) + (riskFreeRate + 0.5 *
          volatility ** 2) * maturity) / (volatility * np.sqrt(maturity))

    return norm.cdf(d1)

In [ ]:
def GammaPutBS(
    spotPrices: np.array,
    strike: float,
    maturity: float,
    riskFreeRate: float,
    volatility: float
) -> np.array:
    """
    Input parameters:
    spotPrices - array of spot prices
    strike - option strike
    maturity - option maturity
    riskFreeRate - current risk free rate 
    volatility - implied BS volatility
    
    Returns:
    np.array of option gammas
    """

    d1 = (np.log(spotPrices / strike) + (riskFreeRate + 0.5 *
          volatility ** 2) * maturity) / (volatility * np.sqrt(maturity))

    return norm.pdf(d1) / (spotPrices * volatility * np.sqrt(maturity))